In [1]:
# 4일차
# CNN은 주변을고려해서 특성
# RNN은 순서적으로 고려해서 특성을 만듦
# http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

In [12]:
import numpy as np
import os
import tensorflow as tf
tf.reset_default_graph() # 그라프 초기화 ( 변수 생성 )
n_inputs = 3  # 입력 데이터
n_neurons = 5 # 셀의 가중치 사이즈
X0 = tf.placeholder(tf.float32,[None,n_inputs]) # 4*3
X1 = tf.placeholder(tf.float32,[None,n_inputs]) # 4*3
# [0,1,2] 사이즈가 하나의 셀로 입력
# FFNN(feed forword neural network)
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) # 가중치 사이즈
# 가중치 사이즈 -> 특성을 찾아내는 것
# static_rnn -> rnn network : 4개의 셀이 연결되면서 메모리 확보
output_seqs, state = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32) 
# 2*4*3 / 배치사이즈 * 셀수 * 뉴런수
Y0,Y1 = output_seqs
# 출력, 다음 셀로 전달되는 값 : 마지막 state값
# (수평으로 셀을 연경해주는 값)
init=tf.global_variables_initializer()
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])


In [13]:
with tf.Session() as sess:
    init.run()
    Y0_val,Y1_val = sess.run([Y0,Y1], feed_dict={X0:X0_batch, X1:X1_batch})

print("처음 데이터 특성 : ", Y0_val,"차수 :", Y0_val.shape)
print("두번째 데이터 특성 : ", Y1_val,"차수 :", Y1_val.shape)
# 4*3 -> 4*5

처음 데이터 특성 :  [[-0.91036344 -0.34917507 -0.33427748  0.7234802   0.93172115]
 [-0.99857163  0.9400998  -0.99443424  0.6079991   0.9989562 ]
 [-0.9999783   0.99908066 -0.9999689   0.45941737  0.9999846 ]
 [ 0.63038296  0.99985343 -0.9999628  -0.9999395  -0.97653687]] 차수 : (4, 5)
두번째 데이터 특성 :  [[-0.9999015   0.9999997  -0.9999995  -0.47917116  0.9998768 ]
 [ 0.6208438   0.7797957  -0.7517404   0.9352928  -0.37674925]
 [-0.9537268   0.99997634 -0.99998343  0.3216574   0.9678108 ]
 [ 0.5986795   0.98501164 -0.9955096  -0.80658996  0.16454647]] 차수 : (4, 5)


In [14]:
 # 셀이 가지고 있는 가중치 사이즈는? 3*5

In [22]:
tf.reset_default_graph() # 그라프 초기화 ( 변수 생성 )

In [23]:
n_steps =28 # sell이 28
n_inputs = 28 # sell당 input size
n_neurons = 150 # 뉴런  150개
n_outputs = 10 # output 10개 확률사이즈
learning_rate = 0.001
# 3차원으로
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # 3차원임
y = tf.placeholder(tf.int32, [None])
# 하나의 셀은 FFNN 임
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32) # 150개의 히든레이어
# 셀 : 28개
# state 마지막 셀의 수평으로 전달되는 값
# state는 마지막 셀의 output과 동일
# 28개의 셀이 있는데 마지막 1개의 output사용 -> many to one (감정분류)-> 분류기
# state의 차수 : 150 * 150 
# output의 차수 : latent time 지연시간을 통해 계산된 셀의
# 모든 값을 결합 출력 150*28*150
# 150개의 특징중에 10개만 출력
logits = tf.layers.dense(states, n_outputs) # 입력차수와 출력차수만 정해주면 자동으로 바이어스 생성
# 가중치 공간을 확보해서 계산해줌
#150 x 150 = 150 x 10
# 가중치 사이즈는 150  10 
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) # 분류를 위한 미분이 가능한식
loss = tf.reduce_mean(xentropy) # 배치 사이즈 -> 평균
optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)# momentum + propgrad
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1) # 양쪽으로 비교해서 큰놈으로
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [24]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [26]:
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size): # 20000*150 돌고 그담에 epoch 만큼 더돌림
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch,
                                            y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch,
                                            y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test,
                                           y:y_test})
        print(epoch, "train accuracy:", acc_train,
             "test acuuracy :", acc_test)

0 train accuracy: 0.94666666 test acuuracy : 0.927
1 train accuracy: 0.94666666 test acuuracy : 0.9437
2 train accuracy: 0.97333336 test acuuracy : 0.9533
3 train accuracy: 0.97333336 test acuuracy : 0.9604
4 train accuracy: 0.9533333 test acuuracy : 0.958
5 train accuracy: 0.98 test acuuracy : 0.9688
6 train accuracy: 0.9866667 test acuuracy : 0.9699
7 train accuracy: 0.9866667 test acuuracy : 0.9713
8 train accuracy: 0.98 test acuuracy : 0.9749
9 train accuracy: 0.99333334 test acuuracy : 0.9739
10 train accuracy: 0.9866667 test acuuracy : 0.9694
11 train accuracy: 0.99333334 test acuuracy : 0.9714
12 train accuracy: 0.9866667 test acuuracy : 0.9717
13 train accuracy: 0.98 test acuuracy : 0.9741
14 train accuracy: 0.97333336 test acuuracy : 0.975
15 train accuracy: 0.96666664 test acuuracy : 0.9752
16 train accuracy: 0.99333334 test acuuracy : 0.9706
17 train accuracy: 0.97333336 test acuuracy : 0.9748
18 train accuracy: 0.98 test acuuracy : 0.9749
19 train accuracy: 0.99333334 test 

In [1]:
import numpy as np
import os
import tensorflow as tf
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

C:\Users\ICT01_22\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_22\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_22\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_22\Anaconda3\envs\tf_test\lib\site-packages

In [4]:
# BasicRNNCell
# MultiRNNCell : 수직으로 레이어 구성
# dynamic_rnn : 모델

reset_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None])
n_neurons = 100
n_layers = 3 # 3개의 멀티 레이어 : 3개의 셀을 생성
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,activation = tf.nn.relu)
         for layer in range(n_layers)]
# 3개의 셀을 조합해서 MultiRNNCell
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
# dynamic_rnn : 셀로 입력 되는 데이터의 개수 맞춰서 셀을 구성
# 나는 학교에 간다. static_rnn 동일한 사이즈 : 큰것 기준.
# 입력사이즈 변동 = 가중치 조절 -> 나가는 특징은 똑같음
# 3층 셀이 28개가 조성 : 고정사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
# state가 몇개가 발생하는가?
# 3 * 150 * 100
states_concat = tf.concat(axis=1, values=states) # 150 *300
logits = tf.layers.dense(states_concat, n_outputs) # 가중치 : 300 * 10
# logits 150*10 / 150은 미니배치 사이즈 / 10개는 확률 특징
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                         logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
init = tf.global_variables_initializer()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.


In [7]:
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size): # 20000*150 돌고 그담에 epoch 만큼 더돌림
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch,
                                            y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch,
                                            y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test,
                                           y:y_test})
        print(epoch, "train accuracy:", acc_train,
             "test acuuracy :", acc_test)
    print("끝 입니다.")

0 train accuracy: 0.91333336 test acuuracy : 0.9332
1 train accuracy: 0.97333336 test acuuracy : 0.9584
2 train accuracy: 0.98 test acuuracy : 0.9709
3 train accuracy: 0.98 test acuuracy : 0.9692
4 train accuracy: 0.98 test acuuracy : 0.9761
5 train accuracy: 0.99333334 test acuuracy : 0.9738
6 train accuracy: 1.0 test acuuracy : 0.9753
7 train accuracy: 0.9866667 test acuuracy : 0.9766
8 train accuracy: 0.98 test acuuracy : 0.9806
9 train accuracy: 0.99333334 test acuuracy : 0.9806
10 train accuracy: 0.99333334 test acuuracy : 0.9787
11 train accuracy: 0.98 test acuuracy : 0.9794
12 train accuracy: 0.9866667 test acuuracy : 0.9769
13 train accuracy: 1.0 test acuuracy : 0.983
14 train accuracy: 0.99333334 test acuuracy : 0.98
15 train accuracy: 0.98 test acuuracy : 0.9804
16 train accuracy: 1.0 test acuuracy : 0.9839
17 train accuracy: 0.98 test acuuracy : 0.9839
18 train accuracy: 0.9866667 test acuuracy : 0.9809
19 train accuracy: 0.99333334 test acuuracy : 0.9827
20 train accuracy: 

In [8]:
#http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128 # 가중치를 갱신하는 단위 배치사이즈
n_input = 28 # 가로 28개
n_step = 28 # 세로 28개 
n_hidden = 128 # 128개의 특성을 추출하겠다. 은닉층의 노드갯수
n_class = 10 # 분류하고자 하는 카테고리수
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 입력층/ ?*가로28*세로28
Y = tf.placeholder(tf.float32, [None, n_class]) # 출력층 ? * 10(카테고리수)
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 가중치 128*10
# 형태가 은닉층의 노드수와 출력층의 노드수를 맞추는것
b = tf.Variable(tf.random_normal([n_class])) # 10
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # neurons : 128
# tf.nn.rnn_cell.BasicRNNCell 이거는 셀만들어주는함수 n_hidden 수만큼 노드생성
# 그 외 GRUCell 등이 있음
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# RNNCell에 의해서 정의된 Recurrent neural network를 생성 즉 
# RNN신경망을 구축해줌
# 여기서 outputs 과 states를 반환받음. 신경망 생성완료
outputs = tf.transpose(outputs, [1,0,2])
# 128*28*128 -> 28*128*128
outputs = outputs[-1]# 28을 날림 128*128
model = tf.matmul(outputs, W) + b # 128*128  + 128*10 -> 128*10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
# 분류를 위한 RNN이기 때문에 softmax를 사용
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 60000 / 128
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch): # 10
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        # _언더바는 파이썬이 계산한 마지막 결과가 저장되는 것
        total_cost += cost_val
    print('Epoch: ', '%04d' % (epoch + 1),
         'Avg cost =', '{:.3f}'.format(total_cost/total_batch))
print('최적화 완료!')

Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch:  0001 Avg cost = 0.503
Epoch:  0002 Avg cost = 0.231
Epoch:  0003 Avg cost = 0.174
Epoch:  0004 Avg cost = 0.146
Epoch:  0005 Avg cost = 0.135
Epoch:  0006 Avg cost = 0.123
Epoch:  0007 Avg cost = 0.122
Epoch:  0008 

In [ ]:
# 문제
1. 테스트 데이터를 이용하여 테스트 하는 회로를 구성하시오
2. 3개의 셀을 갖는 multi layer cell로 수정하시오.


In [ ]:
# BasicRNNCell
# MultiRNNCell : 수직으로 레이어 구성
# dynamic_rnn : 모델
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_neurons = 100
n_layers = 3 #3개의 멀티레이어 (3개의 셀 생성, 3개를 조합해서 MultiRNNCell 만들기)
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]
# 3개의 셀로 멀티레이어를 생성해도 output은 하나의 셀일 경우와 동일함
# state에서는 3개의 레이어마다 
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
# dynamic rnn : 셀로 입력되는 데이터의 개수에 맞춰 셀을 구성
# 입력 사이즈 변동 = 가중치 조절 => 나가는 특징은 일차 (neural 수가 같음)
# static rnn 동일한 사이즈 (큰 것 기준, 작은 것은 padding)
# 3층 셀이 28개 조성 : 고정 사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1,0,2])
#
states_concat = tf.concat(axis=1, values=states) 
logits = tf.layers.dense(states_concat, n_outputs) 
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
#
n_epochs = 100
batch_size = 150

sess = tf.Session() 
init.run()
for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size): # 20000*150 돌고 그담에 epoch 만큼 더돌림
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        # 이미지 사이즈로 생성
        X_batch = X_batch.reshape((-1, n_steps, n_inputs))
        sess.run(training_op, feed_dict={X: X_batch,
                                        y:y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch,
                                        y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:X_test,
                                       y:y_test})
    print(epoch, "train accuracy:", acc_train,
         "test acuuracy :", acc_test)
print("끝 입니다.")

In [ ]:
is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is.correct, tf.float32))
test_batch_size=len(mnist.test.image)
test_xs = mnist.test.image.reshape(test_batch_size,n_step,n_input)
test_ys = mnist.test.labels
print("정확도 : ", sess.run(accuracy, feed_dict={X:test_xs,Y:test_ys}))

In [19]:
values2311 = np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])
print(outputs)

Tensor("rnn/transpose_1:0", shape=(2, 3, 100), dtype=float32)


# LSTM

In [21]:

import numpy as np
import tensorflow as tf
values231 = np.

array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])
print("shape", values231.shape)
import tensorflow as tf
tf.reset_default_graph()
tf_values231 = tf.constant(values231, dtype = tf.float32)
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=100)
# rnn 모델은 동일 : 망과 망을 연결할 때(번역망 : 한글, 영어)
outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                  dtype=tf.float32,inputs=tf_values231)
print(state.c.shape) # 2*100 control state
print(state.h.shape) # 2*100 hidden state
# 이 둘이 연결해줌
# latent time 지연시간
print("rnn이 출력하는 outputs의 의미", outputs.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])

shape (2, 3, 1)
(2, 100)
(2, 100)
rnn이 출력하는 outputs의 의미 (2, 3, 100)


In [23]:
tf.reset_default_graph()
values = tf.constant(np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
]), dtype = tf.float32)
lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
(output_fw, output_bw), (output_state_fw,output_state_bw) = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_cell_fw,cell_bw=lstm_cell_bw,inputs=values,dtype=tf.float32)
# 양방향 LSTM
# Multi cell : 셀에서
print(output_fw.shape)
print(output_bw.shape)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
(2, 3, 100)
(2, 3, 100)


In [27]:
from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops

In [28]:

ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell,x_data,dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape)
    print(states.shape)
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.12575956, -0.38949594],
        [ 0.5346042 , -0.49213177],
        [ 0.070356  , -0.4614423 ],
        [-0.03998384, -0.32661176],
        [ 0.29785615, -0.4337406 ]]], dtype=float32)
(1, 5, 2)
(1, 2)
array([[ 0.29785615, -0.4337406 ]], dtype=float32)


In [49]:
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z']
# 총 26개
num_dic = {n: i for i, n in enumerate(char_arr)}
#  0:a ~ 쭉입력
dic_len = len(num_dic)
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load',
           'love', 'kiss','kind']
# 26 * 26 행렬
# 10000000000000000000000000
# 01000000000000000000000000
# 00100000000000000000000000
def make_batch(seq_data):  # 자동으로 문자를 원핫인코딩
    input_batch = []
    target_batch = []
    for seq in seq_data: # 'word' 부터 순차적으로 가져옴
        input = [num_dic[n] for n in seq[:-1]] # 처음부터 마지막까지 뽑아냄
        # w = 22, o = 14, r = 17 까지만
        target = num_dic[seq[-1]] # d = 3 
        input_batch.append(np.eye(dic_len)[input]) # 원핫 인코딩
        target_batch.append(target)
    return input_batch, target_batch

In [86]:
ops.reset_default_graph()

In [87]:
from tensorflow.contrib import rnn
import tensorflow as tf
from tensorflow.python.framework import ops


learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 특성수 128개
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5, seed=10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 2레이어로 구성된 멀티셀
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell,X,dtype=tf.float32)
# 10 * 3 * 128
# 10 * 128
outputs = tf.transpose(outputs, [1,0,2]) # 3 * 10 * 128
outputs = outputs[-1] # 10 * 128
model = tf.matmul(outputs, W) +b # 비선형 회귀방식
# 분류
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [90]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data) # 단어데이터

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer,cost],feed_dict={X: input_batch, Y:target_batch})
    print("Epoch : ","%04d"%(epoch+1),"cost","{:6f}".format(loss) )
print(acc)


Epoch :  0001 cost 3.664788
Epoch :  0002 cost 2.511354
Epoch :  0003 cost 1.505001
Epoch :  0004 cost 1.259211
Epoch :  0005 cost 0.739402
Epoch :  0006 cost 0.816148
Epoch :  0007 cost 0.386889
Epoch :  0008 cost 0.772344
Epoch :  0009 cost 0.334676
Epoch :  0010 cost 0.312699
Epoch :  0011 cost 0.231489
Epoch :  0012 cost 0.325433
Epoch :  0013 cost 0.245129
Epoch :  0014 cost 0.136524
Epoch :  0015 cost 0.312305
Epoch :  0016 cost 0.032453
Epoch :  0017 cost 0.105486
Epoch :  0018 cost 0.054967
Epoch :  0019 cost 0.040229
Epoch :  0020 cost 0.213527
Epoch :  0021 cost 0.014580
Epoch :  0022 cost 0.057972
Epoch :  0023 cost 0.201457
Epoch :  0024 cost 0.016443
Epoch :  0025 cost 0.011322
Epoch :  0026 cost 0.023502
Epoch :  0027 cost 0.020718
Epoch :  0028 cost 0.139725
Epoch :  0029 cost 0.020285
Epoch :  0030 cost 0.012062


In [98]:
prediction = tf.cast(tf.argmax(model,1),tf.int32)
prediction_check = tf.equal(prediction,Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy],
                               feed_dict={X:input_batch, Y:target_batch})
print(predict)
print(accuracy_val)
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
print("입력값:",[w[:3] + ' ' for w in seq_data])
print("예측값:", predict_words)
print("정확도:", accuracy_val)


[ 3  3 15  4  3 11  3  4 18  3]
1.0
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
